#  Battleship Prototype Page

## Notes:

* Human vs. Computer
* 10 x 10 board (2)
    * "board" is where we place our ships
    * "radar" shows our hits & misses on opponent
* 5 ships
    * Carrier (C, 5)
    * Battleship (B, 4)
    * Submarine (S, 3)
    * Destroyer (D, 3)
    * Patrol Boat (P, 2)
* Human places ships by specifying direction (H / V) and starting coordinate
* Computer randomly generates positions

## Board class

### Attributes
* 10 X 10 matrix
    * list of 10 lists
        * each of these lists has 10 "_" entries
    * On Radar:
        * "*" denotes a hit
        * "@" denotes a miss
    * Board will have ship letters (C,B,S,D,P) show positions
        
### Methods (functions) 
* Mark radar(coordinates) 
* Check coordinates for validity
    * On the board?
    * No prior shot (hit or miss) 
* Display the board

## Ship Class

### Attributes
* Name
* Size
* hit_points
* Row
* Column
* direction (H or V)
* Alive / Dead

### Methods
* is_alive()
* hits_left()
* place_ship(coordinates, direction)
* is_hit(coordinates)

In [34]:
HIT = '*'
MISS = '@'
abbrev = ['C', 'D', 'S', 'B', 'P']
sizes = {
    'C' : 5,
    'B' : 4,
    'D' : 3,
    'S' : 3,
    'P' : 2
}

names = {
    'C' : 'Carrier',
    'B' : 'Battleship',
    'D' : 'Destroyer',
    'S' : 'Submarine',
    'P' : 'Patrol Boat'
}

In [35]:
class Ship:
    def __init__(self, name='', row=0, column=0, direction='h', alive=True):
        self.name = name
        if name != '':
            self.size = sizes[name]
        else:
            self.size = 0
        self.row = row
        self.column = column
        self.alive = alive
        self.direction = direction
        self.hit_points = self.size
        
    def is_alive(self):
        return self.alive
    
    def hits_left(self):
        return self.hit_points
    
    def place_ship(self, row, column, direction):
        self.row = row
        self.column = column
        
    def is_hit(self, row, column):
        if self.direction == 'h':
            if row == self.row and column in range(self.column, self.column + self.size):
                self.hit_points -= 1
                if self.hit_points == 0:
                    self.alive = False
                return True
        if self.direction == 'v':
            if column == self.column and row in range(self.row, self.row + self.size):
                self.hit_points -= 1
                if self.hit_points == 0:
                    self.alive = False
                return True
        
        return False
    

    
            

In [47]:

class Board:
    def __init__(self):
        self.board = []
        self.ships = []
        self.board = [['_' for i in range(10)] for j in range(10)]
        
    def is_valid(self, row, col):
        if row in range(10) and col in range(10):
            if self.board[row][col] not in ([HIT, MISS] + abbrev):
                return True
        return False
    
    def is_clear(self, row, col):
        return self.board[row][col] == '_'
                
    def place_marker(self,abb, row, col, direction=''):
        if abb in [HIT, MISS]:
            self.board[row][col] = abb
        else:
            for i in range(0, sizes[abb]):
                if direction=='v':
                    self.board[row + i][col] = abb
                else:
                    self.board[row][col + i] = abb
                
    def place_ship(self, abb, row, col, direction):
        valid = True
        new_val = 0
        for i in range(0, sizes[abb]):
            if direction == 'h':
                if not self.is_valid(row, col + i):  
                    valid = False
                    break
            else:
                if not self.is_valid(row + i,col): 
                    valid = False
                    break
        if valid:
            ship = Ship(abb, row, col, direction)
            self.place_marker(abb, row, col, direction)
            self.ships.append(ship)
        return valid
    
    def render(self):
        print('    0 1 2 3 4 5 6 7 8 9')
        print('  ---------------------')
        for i, row in enumerate(self.board): # foreach <item> in <container>:
            print(str(i) + ' | ', end='')
            for col in row:
                print(col, end=' ')
            print()
            
    def get_hit_points(self):
        total = 0
        for ship in self.ships:
            total += ship.hits_left()
        return total
    
    def do_shot(self, row, col):
        for ship in self.ships:
            if ship.is_hit(row,col):
                self.place_marker(HIT, row, col)
                return HIT
        self.place_marker(MISS, row, col)
        return MISS

# Player Class

## Attributes
* board - ocean - keeps ships
* board - radar - keeps track of shots (hit, miss)

## Methods
* Place ships
* Take shot (ask other player, ask if you hit or not)
* Determine hit or miss



In [37]:
class Player:
    def __init__(self):
        self.ocean = Board()
        self.radar = Board()
        
    def get_xy(self, coord):
        return int(coord[0]), int(coord[-1])
    
    def place_fleet(self):
        for ship in abbrev: 
            xy = input('Enter coordinates (#,#) for ' + names[ship] + ': ')
            row, col = self.get_xy(xy)
            direction = input('Enter direction (h/v): ')
            while not self.ocean.place_ship(ship, row, col, direction):
                print('Bad Coordinates. Try again.')
                xy = input('Enter coordinates (#,#) for ' + names[ship] + ': ')
                row, col = self.get_xy(xy)
                direction = input('Enter direction (h/v): ')
            self.ocean.render()
            
    def take_shot(self, enemy):
        xy = input('Enter coordinates (#,#) for shot: ')
        row, col = self.get_xy(xy)
        # Note - should REALLY check if valid
        marker = enemy.is_hit(row, col)
        self.radar.place_marker(marker, row, col)
        self.radar.render()
        
    def is_hit(self, row, col):
        marker = self.ocean.do_shot(row, col) # TO DO
        return marker

In [48]:
p = Player()

In [49]:
p.ocean.place_ship('C', 0,0,'h' )

True

In [50]:
p.ocean.render()

    0 1 2 3 4 5 6 7 8 9
  ---------------------
0 | C C C C C _ _ _ _ _ 
1 | _ _ _ _ _ _ _ _ _ _ 
2 | _ _ _ _ _ _ _ _ _ _ 
3 | _ _ _ _ _ _ _ _ _ _ 
4 | _ _ _ _ _ _ _ _ _ _ 
5 | _ _ _ _ _ _ _ _ _ _ 
6 | _ _ _ _ _ _ _ _ _ _ 
7 | _ _ _ _ _ _ _ _ _ _ 
8 | _ _ _ _ _ _ _ _ _ _ 
9 | _ _ _ _ _ _ _ _ _ _ 


In [51]:
p.is_hit(0,0)

'*'

In [52]:
p.ocean.render()

    0 1 2 3 4 5 6 7 8 9
  ---------------------
0 | * C C C C _ _ _ _ _ 
1 | _ _ _ _ _ _ _ _ _ _ 
2 | _ _ _ _ _ _ _ _ _ _ 
3 | _ _ _ _ _ _ _ _ _ _ 
4 | _ _ _ _ _ _ _ _ _ _ 
5 | _ _ _ _ _ _ _ _ _ _ 
6 | _ _ _ _ _ _ _ _ _ _ 
7 | _ _ _ _ _ _ _ _ _ _ 
8 | _ _ _ _ _ _ _ _ _ _ 
9 | _ _ _ _ _ _ _ _ _ _ 


In [53]:
p.is_hit(2,0)

'@'

In [54]:
p.ocean.render()

    0 1 2 3 4 5 6 7 8 9
  ---------------------
0 | * C C C C _ _ _ _ _ 
1 | _ _ _ _ _ _ _ _ _ _ 
2 | @ _ _ _ _ _ _ _ _ _ 
3 | _ _ _ _ _ _ _ _ _ _ 
4 | _ _ _ _ _ _ _ _ _ _ 
5 | _ _ _ _ _ _ _ _ _ _ 
6 | _ _ _ _ _ _ _ _ _ _ 
7 | _ _ _ _ _ _ _ _ _ _ 
8 | _ _ _ _ _ _ _ _ _ _ 
9 | _ _ _ _ _ _ _ _ _ _ 


In [ ]:
board.place_ship('D', 1, 0, 'v')

In [ ]:
board.render()

In [ ]:
board.ships[0].row

In [ ]:
range(4)